# Data Loading (Python)
<small>Tutorial written by: Hannah-Marie Martiny (hanmar@food.dtu.dk)
<br>Last updated: 20-12-2021</small>

In this tutorial, a brief overview of how to load the data in the different formats are given: MySQL, TSV and HDF.

<em>NOTE: The various settings that are sensitive are stored in a config.json file, but just change the settings to what fit your own setup.</em>

In [10]:
import json
with open('../config.json', 'r') as source:
    config = json.load(source)

database=config['database'] # name of database
host=config['host'] # host address of MySQL server
port=config['port'] # port of MySQL server
user=config['user'] # user name
passwd=config['password'] # password for user 
dataDir=config['datadir'] # directory where data files are stored

 ## MySQL
### Loading MySQL dumps
For each of the dump files (.sql), load them with the command `mysql db_name < dump-file.sql`. `db_name` is the name of database to contain the tables, and the `dump-file.sql` is the file containing the table structure and data.

We have written a short bash script [`loader.sh`](loader.sh) for this, where we give the path to the directory containing dump files (`-d $dataDir`) and the name of the MySQL database (`-n $database`):
```
> ../loader.sh -d $dataDir -n $database
```

### Reading data from a MySQL  database
#### Pandas 
It is possibly to use the [`pandas.read_sql`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) function to query the database, although this also requires [mysql-connector-python](https://dev.mysql.com/doc/connector-python/en/connector-python-installation-binary.html) to be installed.

In [7]:
!python -m pip install mysql-connector-python
import mysql.connector as connection
import pandas as pd

mydb = connection.connect(host=host, port=port, database=database, user=user, passwd=passwd, use_pure=True)
query = "select * from metadata"
df = pd.read_sql(query, mydb)
mydb.close()

df.head()

You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


,run_accession,sample_accession,project_accession,country,location,continent,collection_date,tax_id,host,host_tax_id,instrument_platform,instrument_model,library_layout,raw_reads,trimmed_reads,raw_bases,trimmed_bases,trimmed_fragments
0,DRR000836,SAMD00002573,PRJDA61421,None,None,None,None,939928,rhizosphere metagenome,NaN,LS454,454 GS FLX Titanium,SINGLE,1268608,1247751.0,641025182,411961081.0,1247751.0
1,DRR000980,SAMD00010106,PRJDB2325,None,None,None,None,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,1207522,1190673.0,596228115,416966759.0,1190673.0
2,DRR000981,SAMD00010105,PRJDB2325,None,None,None,None,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,802422,792888.0,424054817,303648013.0,792888.0
3,DRR001376,SAMD00006238,PRJDA72837,None,None,None,None,9606,Homo sapiens,9606.0,ILLUMINA,Illumina Genome Analyzer IIx,SINGLE,336278,256975.0,42034750,26464638.0,256975.0
4,DRR001455,SAMD00015677,PRJDB2729,None,None,None,None,410658,soil metagenome,NaN,ILLUMINA,Illumina Genome Analyzer IIx,PAIRED,21452087,5146184.0,3217813050,336410211.0,7991359.0


#### Custom function that uses MySQL cli
This is a custom function that interacts with the mysql client installed in your terminal, where you can add the configuration settings in the `args=` argument in the function.

In [6]:
import subprocess
import pandas as pd
from io import StringIO

def query_db(query, args=''):
    cmd = "mysql {} -e \"{}\"".format(args, query)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if p.returncode > 0:
        print("Failed to query database with error:")
        print(p.stderr.decode())
    
    else:
        df = pd.read_csv(StringIO(p.stdout.decode()), sep='\t')
        return df

cli_args = f"--database={database} --host={host} --port={port} --user={user} --password={passwd}"
df = query_db("select * from metadata", args=cli_args)
df.head()

,run_accession,sample_accession,project_accession,country,location,continent,collection_date,tax_id,host,host_tax_id,instrument_platform,instrument_model,library_layout,raw_reads,trimmed_reads,raw_bases,trimmed_bases,trimmed_fragments
0,DRR000836,SAMD00002573,PRJDA61421,NaN,NaN,NaN,NaN,939928,rhizosphere metagenome,NaN,LS454,454 GS FLX Titanium,SINGLE,1268608,1247751.0,641025182,411961081.0,1247751.0
1,DRR000980,SAMD00010106,PRJDB2325,NaN,NaN,NaN,NaN,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,1207522,1190673.0,596228115,416966759.0,1190673.0
2,DRR000981,SAMD00010105,PRJDB2325,NaN,NaN,NaN,NaN,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,802422,792888.0,424054817,303648013.0,792888.0
3,DRR001376,SAMD00006238,PRJDA72837,NaN,NaN,NaN,NaN,9606,Homo sapiens,9606.0,ILLUMINA,Illumina Genome Analyzer IIx,SINGLE,336278,256975.0,42034750,26464638.0,256975.0
4,DRR001455,SAMD00015677,PRJDB2729,NaN,NaN,NaN,NaN,410658,soil metagenome,NaN,ILLUMINA,Illumina Genome Analyzer IIx,PAIRED,21452087,5146184.0,3217813050,336410211.0,7991359.0


## Tab-separated files (TSV)
It is fairly straightforward to read the .tsv files with Pandas, here is an example:

In [8]:
import os
import pandas as pd
tsvFile = os.path.join(dataDir, 'metadata.tsv')
df = pd.read_csv(tsvFile, sep='\t')
df.head()

,run_accession,sample_accession,project_accession,country,location,continent,collection_date,tax_id,host,host_tax_id,instrument_platform,instrument_model,library_layout,raw_reads,trimmed_reads,raw_bases,trimmed_bases,trimmed_fragments
0,DRR000836,SAMD00002573,PRJDA61421,NaN,NaN,NaN,NaN,939928,rhizosphere metagenome,NaN,LS454,454 GS FLX Titanium,SINGLE,1268608,1247751,641025182,411961081,1247751
1,DRR000980,SAMD00010106,PRJDB2325,NaN,NaN,NaN,NaN,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,1207522,1190673,596228115,416966759,1190673
2,DRR000981,SAMD00010105,PRJDB2325,NaN,NaN,NaN,NaN,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,802422,792888,424054817,303648013,792888
3,DRR001376,SAMD00006238,PRJDA72837,NaN,NaN,NaN,NaN,9606,Homo sapiens,9606.0,ILLUMINA,Illumina Genome Analyzer IIx,SINGLE,336278,256975,42034750,26464638,256975
4,DRR001455,SAMD00015677,PRJDB2729,NaN,NaN,NaN,NaN,410658,soil metagenome,NaN,ILLUMINA,Illumina Genome Analyzer IIx,PAIRED,21452087,5146184,3217813050,336410211,7991359


## HDF5 files
HDF5 is a data software ibrary that is built for fast I/O processing and storage, and the pandas library contains the function [`pandas.read_hdf`](https://pandas.pydata.org/docs/reference/api/pandas.read_hdf.html).

In [9]:
import os
import pandas as pd
h5File = os.path.join(dataDir, 'metadata.h5')
df = pd.read_hdf(h5File)
df.head()

,run_accession,sample_accession,project_accession,country,location,continent,collection_date,tax_id,host,host_tax_id,instrument_platform,instrument_model,library_layout,raw_reads,trimmed_reads,raw_bases,trimmed_bases,trimmed_fragments
0,DRR000836,SAMD00002573,PRJDA61421,NaN,NaN,NaN,NaN,939928,rhizosphere metagenome,NaN,LS454,454 GS FLX Titanium,SINGLE,1268608,1247751,641025182,411961081,1247751
1,DRR000980,SAMD00010106,PRJDB2325,NaN,NaN,NaN,NaN,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,1207522,1190673,596228115,416966759,1190673
2,DRR000981,SAMD00010105,PRJDB2325,NaN,NaN,NaN,NaN,1006967,shoot metagenome,NaN,LS454,454 GS FLX,SINGLE,802422,792888,424054817,303648013,792888
3,DRR001376,SAMD00006238,PRJDA72837,NaN,NaN,NaN,NaN,9606,Homo sapiens,9606.0,ILLUMINA,Illumina Genome Analyzer IIx,SINGLE,336278,256975,42034750,26464638,256975
4,DRR001455,SAMD00015677,PRJDB2729,NaN,NaN,NaN,NaN,410658,soil metagenome,NaN,ILLUMINA,Illumina Genome Analyzer IIx,PAIRED,21452087,5146184,3217813050,336410211,7991359
